In [58]:
import requests
from dotenv import load_dotenv
import os
load_dotenv
API_KEY = os.getenv("API_KEY")


In [31]:
import steamreviews

request_params = dict()
request_params['language'] = 'english'
request_params['review_type'] = 'positive'
request_params['purchase_type'] = 'steam'
request_params['day_range'] = '10'


app_id = 730
review_dict, query_count = steamreviews.download_reviews_for_app_id(app_id,
                                                                    chosen_request_params=request_params)


[appID = 730] expected #reviews = 1073495


In [43]:
players = []
for key in review_dict["reviews"].keys():
    if len(players) < 10:
        players.append(review_dict["reviews"][key]["author"]["steamid"])

In [55]:
# For every player, get their friends
friends = []
for player in players:
    friends.append(player)
    url = f'http://api.steampowered.com/ISteamUser/GetFriendList/v0001/?key={API_KEY}&steamid={player}&relationship=friend'
    response = requests.get(url)
    friend_list = response.json()
    try:
        for item in friend_list["friendslist"]["friends"]:
            friends.append(item["steamid"])
    except:
        continue

In [57]:
# for every friend, get their games
friends_with_game = []
for friend in friends:
    games = []
    url = f'http://api.steampowered.com/IPlayerService/GetOwnedGames/v0001/?key={API_KEY}&steamid={friend}&format=json'
    response = requests.get(url)
    game_list = response.json()
    try:
        for item in game_list["response"]["games"]:
            games.append(item["appid"])
    except:
        continue
    if app_id in games:
        friends_with_game.append(friend)
        

73

183